In [73]:
import pandas as pd
import yfinance as yf
import pickle
from datetime import date
from dateutil.relativedelta import relativedelta
from os.path import exists
import sqlite3

In [105]:
tickers = ['JPM', 'TSLA', 'SPY', 'BRK-B', 'INTC', 'SSO', 'ROM', 'AAPL', 'GME']

In [122]:
# today's date + 1 for the end date
end_dt = date.today() + relativedelta(days=+1)

# setup sql connection
#engine = create_engine('sqlite:///data/stocks.db', echo=False)
con = sqlite3.connect('data/stocks.db')

# create my tables if they don't exist
con.execute('create table if not exists stocks ( date text, open float, high float, low float, close float, adj_close float, volume float, ticker text );')
con.execute('create index if not exists "ix_stocks_ticker" ON stocks ("ticker");')
con.execute('create index if not exists "ix_stocks_ticker_sym" ON stocks ("ticker", "sym");')

con.execute('create table if not exists stocks_upload ( date text, open float, high float, low float, close float, adj_close float, volume float, ticker text );')
con.execute('create index if not exists "ix_stocks_upload_ticker" ON stocks_upload ("ticker");')
con.execute('create index if not exists "ix_stocks_upload_ticker_sym" ON stocks_upload ("ticker", "sym");')

In [123]:
# get max dates from stocks.db
max_date = {}
for row in con.execute('SELECT ticker, max(Date) FROM stocks group by ticker ORDER BY ticker'):
    max_date[ row[0] ] = row[1].split()[0]

# loop through each ticker
for sym in tickers:
    
    # setting start date based off the max date in the database
    # always assuming the last entry in the database is a partial day 
    # pull so pulling that again. Otherwise set to 2000
    if sym not in max_date:
        start_dt = '2000-01-01'
    else:
        start_dt = max_date[ sym ]
    
    print( sym + ' ' + str(start_dt) + ' to ' + str(end_dt) )
    
    # download the data we need
    df = yf.download(sym,
              start=start_dt, 
              end=end_dt, 
              progress=False)
    if len( df_prev ) > 0:        
        # merge the dataframes
        df_index = df.index
        df_prev_index = df_prev.index
        mask = ~df_prev_index.isin(df_index)
        #result = pd_prev.loc[mask]
        df = pd.concat([df_prev.loc[mask],df])

    # add the ticker to the df for prep to send to DB
    df['ticker'] = sym

    # fix some column names like I like
    df = df.reset_index()                             # move Date to regular column
    df.columns = df.columns.str.lower()               # lower case column names
    df.columns = df.columns.str.replace(' ', '_')     # replace spaces in names with _
    
    # saving to temp table. then will delete overlapping data from base table
    df.to_sql('stocks_upload', con=con, if_exists='replace', index=False ) #, dtype={'date': 'text'} )
    
    # delete from base table where we have over lapping data
    sql = 'delete from stocks where ticker like "' + sym + '" and date in (select date from stocks_upload group by date)'
    con.execute(sql)
    
    # add the newly updated data to the base table
    con.execute("insert into stocks select * from stocks_upload")
    
    # save changes
    con.commit()

# close connection to sqlite
con.close()

JPM 2000-01-01 to 2022-08-06
TSLA 2000-01-01 to 2022-08-06
SPY 2000-01-01 to 2022-08-06
BRK-B 2000-01-01 to 2022-08-06
INTC 2000-01-01 to 2022-08-06
SSO 2000-01-01 to 2022-08-06
ROM 2000-01-01 to 2022-08-06
AAPL 2000-01-01 to 2022-08-06
GME 2000-01-01 to 2022-08-06


In [124]:
df

,date,open,high,low,close,adj_close,volume,ticker
0,2002-02-13,2.406250,2.515000,2.381250,2.512500,1.691667,76216000,GME
1,2002-02-14,2.543750,2.548750,2.481250,2.500000,1.683250,11021600,GME
2,2002-02-15,2.500000,2.506250,2.462500,2.487500,1.674834,8389600,GME
3,2002-02-19,2.475000,2.475000,2.343750,2.387500,1.607504,7410400,GME
4,2002-02-20,2.400000,2.468750,2.381250,2.468750,1.662210,6892800,GME
...,...,...,...,...,...,...,...,...
5151,2022-08-01,33.799999,35.709999,33.770000,34.779999,34.779999,4196600,GME
5152,2022-08-02,35.900002,36.799999,35.119999,35.840000,35.840000,4697100,GME
5153,2022-08-03,36.220001,38.009998,36.000000,37.930000,37.930000,5186500,GME
5154,2022-08-04,38.340000,39.889999,37.889999,38.360001,38.360001,5421500,GME
